In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5

In [ ]:
from tensorflow import keras
import tensorflow as tf

In [ ]:
model = keras.models.load_model("dino_dragon_10_0.899.h5")

In [ ]:
model.summary()

In [ ]:
model.inputs

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
import numpy as np

In [ ]:
dino_img = load_img("test/dino/0c494627-2ed2-49c7-9ff7-e70308ad5fcb.jpg", target_size=(150, 150))
dragon_img = load_img("test/dragon/01f69481-dab1-41f0-9c69-01d1969aa575.jpg", target_size=(150, 150))

In [ ]:
X_dino = np.array([np.array(dino_img)/255])
X_dragon = np.array([np.array(dragon_img)/255])

In [ ]:
dino_prediction = model.predict(X_dino)
dragon_prediction = model.predict(X_dragon)

In [ ]:
dragon_prediction[0, 0]

In [ ]:
dino_prediction[0, 0]

## Convert to Lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('reptile-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

### Question 1

In [ ]:
!ls -lh

In [ ]:
import tensorflow.lite as tflite

In [ ]:
interpreter = tflite.Interpreter(model_path='reptile-model.tflite')
interpreter.allocate_tensors()

In [ ]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

### Question 2

In [ ]:
input_index, output_index

In [ ]:
interpreter.set_tensor(input_index, np.array(X_dragon, dtype=np.float32))

In [ ]:
interpreter.invoke()

In [ ]:
interpreter.get_tensor(output_index)

## Obtaining image

In [ ]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [ ]:
img = download_image("https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg")

In [ ]:
img = prepare_image(img, (150, 150))

In [ ]:
img

### Question 3

In [ ]:
X = np.array([np.array(img)/255])
X[0, 0, 0, 0]

### Question 4

In [ ]:
interpreter.set_tensor(input_index,  np.array(X, dtype=np.float32))
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds